# Concurrency Utilities - Core Async Helpers

The `concurrency._utils` module provides essential utilities for async/await workflows in lionherd:

**Core Functions:**
- **is_coro_func**: Cached check for coroutine functions (handles partials)
- **current_time**: Monotonic clock time for performance measurement
- **run_sync**: Run blocking functions in thread pool without blocking event loop
- **sleep**: Async sleep without blocking event loop

**Key Benefits:**
- Type-safe async utilities with proper ParamSpec support
- Cached coroutine detection for performance
- anyio-based for compatibility across asyncio/trio
- Thread pool execution for blocking operations

In [1]:
import time
from functools import partial

from lionherd_core.libs.concurrency import (
    current_time,
    gather,
    is_coro_func,
    run_sync,
    sleep,
)

## 1. is_coro_func - Coroutine Function Detection

Check if a callable is a coroutine function. Handles edge cases like `functools.partial`.

In [2]:
# Regular function
def sync_func():
    return "sync"


# Async function
async def async_func():
    return "async"


print(f"sync_func is coroutine: {is_coro_func(sync_func)}")
print(f"async_func is coroutine: {is_coro_func(async_func)}")

sync_func is coroutine: False
async_func is coroutine: True


In [3]:
# Handles partials correctly
async def async_with_args(x, y):
    return x + y


partial_async = partial(async_with_args, 10)

print(f"partial of async func is coroutine: {is_coro_func(partial_async)}")
print(f"Result: {await partial_async(5)}")

partial of async func is coroutine: True
Result: 15


**Caching Benefit**: The check is cached via `@cache`, so repeated calls with the same function are instant.

In [4]:
# First call - computes and caches
start = time.perf_counter()
for _ in range(10000):
    is_coro_func(async_func)
elapsed = time.perf_counter() - start

print(f"10,000 cached checks: {elapsed * 1000:.3f}ms")
print(f"Per check: {elapsed * 1000000 / 10000:.3f}µs")

10,000 cached checks: 0.762ms
Per check: 0.076µs


## 2. current_time - Monotonic Clock

Get current time from monotonic clock. Used for performance measurement and timeouts.

In [5]:
# Get current time
t1 = current_time()
print(f"Current time: {t1:.6f}s")

# Measure elapsed time
await sleep(0.1)
t2 = current_time()
elapsed = t2 - t1

print(f"Elapsed: {elapsed:.3f}s")
print(f"Close to 0.1s: {abs(elapsed - 0.1) < 0.01}")

Current time: 1015970.893361s
Elapsed: 0.102s
Close to 0.1s: True


**Why Monotonic?**
- Not affected by system clock changes
- Always moves forward
- Perfect for measuring intervals and timeouts

## 3. run_sync - Thread Pool Execution

Run blocking synchronous functions in a thread pool without blocking the event loop. Preserves type hints via ParamSpec.

In [6]:
# Blocking function that would freeze event loop
def blocking_operation(duration: float, name: str) -> str:
    time.sleep(duration)  # Simulates blocking I/O
    return f"Completed {name} after {duration}s"


# Run in thread pool - event loop stays responsive
result = await run_sync(blocking_operation, 0.2, name="task-1")
print(result)

Completed task-1 after 0.2s


In [7]:
# Multiple blocking calls in parallel
async def parallel_blocking():
    start = current_time()

    # All run in thread pool concurrently
    results = await gather(
        run_sync(blocking_operation, 0.15, name="task-A"),
        run_sync(blocking_operation, 0.15, name="task-B"),
        run_sync(blocking_operation, 0.15, name="task-C"),
    )

    elapsed = current_time() - start
    return results, elapsed


results, total_time = await parallel_blocking()
print(f"Results: {results}")
print(f"Total time: {total_time:.3f}s")
print(f"Parallel execution: {total_time < 0.3}")

Results: ['Completed task-A after 0.15s', 'Completed task-B after 0.15s', 'Completed task-C after 0.15s']
Total time: 0.156s
Parallel execution: True


**Type Safety**: `run_sync` uses `ParamSpec` to preserve function signature.

In [8]:
# Function with specific signature
def compute(x: int, y: int, operation: str = "add") -> int:
    if operation == "add":
        return x + y
    elif operation == "multiply":
        return x * y
    raise ValueError(f"Unknown operation: {operation}")


# Keyword arguments work correctly
result1 = await run_sync(compute, 10, 5, operation="add")
result2 = await run_sync(compute, 10, 5, operation="multiply")

print(f"10 + 5 = {result1}")
print(f"10 * 5 = {result2}")

10 + 5 = 15
10 * 5 = 50


## 4. sleep - Async Sleep

Sleep without blocking the event loop. Essential for delays in async workflows.

In [9]:
# Simple delay
print("Starting...")
await sleep(0.1)
print("Finished after 0.1s")

Starting...
Finished after 0.1s


In [10]:
# Event loop stays responsive during sleep
async def countdown(n: int):
    for i in range(n, 0, -1):
        print(f"Countdown: {i}")
        await sleep(0.05)
    print("Done!")


async def other_task():
    for i in range(3):
        await sleep(0.08)
        print(f"  Other task tick {i + 1}")


# Both run concurrently
await gather(countdown(3), other_task())

Countdown: 3
Countdown: 2
  Other task tick 1
Countdown: 1
Done!
  Other task tick 2
  Other task tick 3


[None, None]

## 5. Practical Example - Rate Limiter

Combine utilities to build a simple rate limiter.

In [11]:
class RateLimiter:
    """Simple rate limiter using concurrency utilities."""

    def __init__(self, max_calls: int, period: float):
        self.max_calls = max_calls
        self.period = period
        self.calls: list[float] = []

    async def acquire(self):
        """Wait until call is allowed."""
        now = current_time()

        # Remove old calls outside window
        self.calls = [t for t in self.calls if now - t < self.period]

        # Wait if at limit
        if len(self.calls) >= self.max_calls:
            wait_time = self.period - (now - self.calls[0])
            print(f"Rate limit hit, waiting {wait_time:.3f}s")
            await sleep(wait_time)
            self.calls = self.calls[1:]  # Remove oldest

        self.calls.append(current_time())


# Test: 3 calls per 0.5s
limiter = RateLimiter(max_calls=3, period=0.5)

start = current_time()
for i in range(5):
    await limiter.acquire()
    print(f"Call {i + 1} at {current_time() - start:.3f}s")

total = current_time() - start
print(f"\nTotal time: {total:.3f}s")

Call 1 at 0.000s
Call 2 at 0.000s
Call 3 at 0.000s
Rate limit hit, waiting 0.500s
Call 4 at 0.501s
Call 5 at 0.501s

Total time: 0.502s


## 6. Practical Example - Performance Monitor

Track execution time of blocking operations.

In [12]:
async def monitor_blocking_task(func, *args, **kwargs):
    """Run blocking task and measure performance."""
    start = current_time()

    # Detect if function is async or sync
    if is_coro_func(func):
        result = await func(*args, **kwargs)
        execution_type = "async"
    else:
        result = await run_sync(func, *args, **kwargs)
        execution_type = "sync (threaded)"

    elapsed = current_time() - start

    return {
        "result": result,
        "elapsed": elapsed,
        "type": execution_type,
    }


# Test with sync function
def heavy_computation(n: int) -> int:
    time.sleep(0.1)  # Simulate work
    return sum(range(n))


# Test with async function
async def light_async_work(n: int) -> int:
    await sleep(0.1)
    return n * 2


# Monitor both
result1 = await monitor_blocking_task(heavy_computation, 1000)
print(f"Sync task: {result1['result']} in {result1['elapsed']:.3f}s ({result1['type']})")

result2 = await monitor_blocking_task(light_async_work, 42)
print(f"Async task: {result2['result']} in {result2['elapsed']:.3f}s ({result2['type']})")

Sync task: 499500 in 0.106s (sync (threaded))
Async task: 84 in 0.101s (async)


## Summary Checklist

**Concurrency Utilities Essentials:**
- ✅ `is_coro_func`: Cached coroutine detection (handles partials)
- ✅ `current_time`: Monotonic clock for intervals and timeouts
- ✅ `run_sync`: Thread pool execution with full type safety
- ✅ `sleep`: Async sleep without blocking event loop
- ✅ anyio-based for asyncio/trio compatibility
- ✅ ParamSpec support preserves function signatures
- ✅ Efficient caching for repeated checks

**Common Patterns:**
- Rate limiting with `current_time` + `sleep`
- Performance monitoring with time measurement
- Parallel blocking operations via `run_sync` + `gather`
- Dynamic async/sync handling with `is_coro_func`

**Next Steps:**
- See `Throttle` for advanced rate limiting
- See `Broadcaster` for pub/sub patterns
- See `Flow` for orchestrating concurrent operations